<a href="https://colab.research.google.com/github/tallywiesenberg/DS-Unit-1-Sprint-3-Statistical-Tests-and-Experiments/blob/master/Wiesenberg_LS_DS_143_Introduction_to_Bayesian_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lambda School Data Science Module 143

## Introduction to Bayesian Inference

!['Detector! What would the Bayesian statistician say if I asked him whether the--' [roll] 'I AM A NEUTRINO DETECTOR, NOT A LABYRINTH GUARD. SERIOUSLY, DID YOUR BRAIN FALL OUT?' [roll] '... yes.'](https://imgs.xkcd.com/comics/frequentists_vs_bayesians.png)

*[XKCD 1132](https://www.xkcd.com/1132/)*


## Prepare - Bayes' Theorem and the Bayesian mindset

Bayes' theorem possesses a near-mythical quality - a bit of math that somehow magically evaluates a situation. But this mythicalness has more to do with its reputation and advanced applications than the actual core of it - deriving it is actually remarkably straightforward.

### The Law of Total Probability

By definition, the total probability of all outcomes (events) if some variable (event space) $A$ is 1. That is:

$$P(A) = \sum_n P(A_n) = 1$$

The law of total probability takes this further, considering two variables ($A$ and $B$) and relating their marginal probabilities (their likelihoods considered independently, without reference to one another) and their conditional probabilities (their likelihoods considered jointly). A marginal probability is simply notated as e.g. $P(A)$, while a conditional probability is notated $P(A|B)$, which reads "probability of $A$ *given* $B$".

The law of total probability states:

$$P(A) = \sum_n P(A | B_n) P(B_n)$$

In words - the total probability of $A$ is equal to the sum of the conditional probability of $A$ on any given event $B_n$ times the probability of that event $B_n$, and summed over all possible events in $B$.

### The Law of Conditional Probability

What's the probability of something conditioned on something else? To determine this we have to go back to set theory and think about the intersection of sets:

The formula for actual calculation:

$$P(A|B) = \frac{P(A \cap B)}{P(B)}$$

![Visualization of set intersection](https://upload.wikimedia.org/wikipedia/commons/9/99/Venn0001.svg)

Think of the overall rectangle as the whole probability space, $A$ as the left circle, $B$ as the right circle, and their intersection as the red area. Try to visualize the ratio being described in the above formula, and how it is different from just the $P(A)$ (not conditioned on $B$).

We can see how this relates back to the law of total probability - multiply both sides by $P(B)$ and you get $P(A|B)P(B) = P(A \cap B)$ - replaced back into the law of total probability we get $P(A) = \sum_n P(A \cap B_n)$.

This may not seem like an improvement at first, but try to relate it back to the above picture - if you think of sets as physical objects, we're saying that the total probability of $A$ given $B$ is all the little pieces of it intersected with $B$, added together. The conditional probability is then just that again, but divided by the probability of $B$ itself happening in the first place.

### Bayes Theorem

Here is is, the seemingly magic tool:

$$P(A|B) = \frac{P(B|A)P(A)}{P(B)}$$

In words - the probability of $A$ conditioned on $B$ is the probability of $B$ conditioned on $A$, times the probability of $A$ and divided by the probability of $B$. These unconditioned probabilities are referred to as "prior beliefs", and the conditioned probabilities as "updated."

Why is this important? Scroll back up to the XKCD example - the Bayesian statistician draws a less absurd conclusion because their prior belief in the likelihood that the sun will go nova is extremely low. So, even when updated based on evidence from a detector that is $35/36 = 0.972$ accurate, the prior belief doesn't shift enough to change their overall opinion.

There's many examples of Bayes' theorem - one less absurd example is to apply to [breathalyzer tests](https://www.bayestheorem.net/breathalyzer-example/). You may think that a breathalyzer test that is 100% accurate for true positives (detecting somebody who is drunk) is pretty good, but what if it also has 8% false positives (indicating somebody is drunk when they're not)? And furthermore, the rate of drunk driving (and thus our prior belief)  is 1/1000.

What is the likelihood somebody really is drunk if they test positive? Some may guess it's 92% - the difference between the true positives and the false positives. But we have a prior belief of the background/true rate of drunk driving. Sounds like a job for Bayes' theorem!

$$
\begin{aligned}
P(Drunk | Positive) &= \frac{P(Positive | Drunk)P(Drunk)}{P(Positive)} \\
&= \frac{1 \times 0.001}{0.08} \\
&= 0.0125
\end{aligned}
$$

In other words, the likelihood that somebody is drunk given they tested positive with a breathalyzer in this situation is only 1.25% - probably much lower than you'd guess. This is why, in practice, it's important to have a repeated test to confirm (the probability of two false positives in a row is $0.08 * 0.08 = 0.0064$, much lower), and Bayes' theorem has been relevant in court cases where proper consideration of evidence was important.

## Live Lecture - Deriving Bayes' Theorem, Calculating Bayesian Confidence

Notice that $P(A|B)$ appears in the above laws - in Bayesian terms, this is the belief in $A$ updated for the evidence $B$. So all we need to do is solve for this term to derive Bayes' theorem. Let's do it together!

In [3]:
# Activity 2 - Use SciPy to calculate Bayesian confidence intervals
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.bayes_mvs.html#scipy.stats.bayes_mvs
from scipy import stats
import numpy as np

#set a random seed
np.random.seed(seed=10)

coinflips = np.random.binomial(1, 0.5, 100)
coinflips

array([1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0,
       0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1])

In [4]:
#frequentist approach
def confidence_interval(data, confidence=0.95):
  n = len(data)
  mean = sum(data)/n
  stderr = stats.sem(data)
  interval = stderr * stats.t.ppf(1 + confidence / 2.0, n-1)
  return(mean, mean - interval, mean + interval)
confidence_interval(coinflips, 0.99)

(0.49, nan, nan)

##Bayesian Modeling with PyMC3

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
import pymc3 as pm
import scipy.stats

In [0]:
def display_probs(d):
  for key, value in d.items():
    print(f'Species: {key:8} Prevalence: {100*value:.2f}%.')

In [0]:
#observations
animals = ['lions', 'tigers', 'bears']
c = np.array([3, 2, 1])
#observations
alpha = np.array([1, 1, 1])


## Assignment - Code it up!

Most of the above was pure math - now write Python code to reproduce the results! This is purposefully open ended - you'll have to think about how you should represent probabilities and events. You can and should look things up, and as a stretch goal - refactor your code into helpful reusable functions!

Specific goals/targets:

1. Write a function `def prob_drunk_given_positive(prob_drunk_prior, prob_positive, prob_positive_drunk)` that reproduces the example from lecture, and use it to calculate and visualize a range of situations
2. Explore `scipy.stats.bayes_mvs` - read its documentation, and experiment with it on data you've tested in other ways earlier this week
3. Create a visualization comparing the results of a Bayesian approach to a traditional/frequentist approach
4. In your own words, summarize the difference between Bayesian and Frequentist statistics

If you're unsure where to start, check out [this blog post of Bayes theorem with Python](https://dataconomy.com/2015/02/introduction-to-bayes-theorem-with-python/) - you could and should create something similar!

Stretch goals:

- Apply a Bayesian technique to a problem you previously worked (in an assignment or project work) on from a frequentist (standard) perspective
- Check out [PyMC3](https://docs.pymc.io/) (note this goes beyond hypothesis tests into modeling) - read the guides and work through some examples
- Take PyMC3 further - see if you can build something with it!

In [0]:
#situation specific function
def prob_drunk_given_positive(prob_drunk_prior, prob_positive, prob_positive_drunk):
  prob = prob_drunk_prior*prob_positive_drunk/prob_positive
  return prob

In [9]:
prob_drunk_given_positive(0.005, 0.01, 1)

0.5

In [0]:
#generalizable bayesian probability function
def prob_a_given_b(prob_a, prob_b, prob_b_given_a):
  prob = prob_drunk_prior*prob_positive_drunk/prob_positive
  return prob

In [0]:
###CONFIDENCE/BAYESIAN INTERVALS

#load data
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data',
                 na_values='?', header=None, names=['party', 
                                                                                                                                     'handicapped-infants',
                                                                                                                                     ' water-project-cost-sharing',
                                                                                                                                     'adoption-of-the-budget-resolution',
                                                                                                                                     'physician-fee-freeze',
                                                                                                                                     'el-salvador-aid',
                                                                                                                                     'religious-groups-in-schools',
                                                                                                                                     'anti-satellite-test-ban',
                                                                                                                                     'aid-to-nicaraguan-contras',
                                                                                                                                     'mx-missile',
                                                                                                                                     'immigration',
                                                                                                                                     'synfuels-corporation-cutback',
                                                                                                                                     'education-spending',
                                                                                                                                     'superfund-right-to-sue',
                                                                                                                                     'crime',
                                                                                                                                     'duty-free-exports',
                                                                                                                                     'export-administration-act-south-africa'])


#impute with mode
for col in df.columns[df.columns != 'party']:
  df[col] = df[col].fillna(value=df[col].mode()[0]) #have to include [0] because mode returns a series
  #map y/n to 0/1
  d = {'n': 0, 'y': 1}
  df[col] = df[col].map(d)

In [18]:
df.head()

,party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,0,1,0,1,1,1,0,0,0,1,0,1,1,1,0,1
1,republican,0,1,0,1,1,1,0,0,0,0,0,1,1,1,0,1
2,democrat,0,1,1,0,1,1,0,0,0,0,1,0,1,1,0,0
3,democrat,0,1,1,0,1,1,0,0,0,0,1,0,1,0,0,1
4,democrat,1,1,1,0,1,1,0,0,0,0,1,0,1,1,1,1


In [0]:
##calculate confidence interval: frequentist approach
  #load function
def confidence_interval(data, confidence=0.95):
  """
  Calculate a confidence interval around a sample mean for given data.
  Using t-distribution and two-tailed test, default 95% confidence. 
  
  Arguments:
    data - iterable (list or numpy array) of sample observations
    confidence - level of confidence for the interval
  
  Returns:
    tuple of (mean, lower bound, upper bound)
  """
  data = np.array(data)
  mean = np.mean(data)
  n = len(data)
  stderr = stats.sem(data)
  interval = stderr * stats.t.ppf((1 + confidence) / 2.0, n - 1)
  high_estimate = mean + interval
  low_estimate = mean - interval
  print(f'Mean: {mean}')
  print(f'Low Estimate: {high_estimate}')
  print(f'High Estimate: {low_estimate}')
  return (mean, low_estimate, high_estimate)

In [21]:
  #run confidence interval
confidence_interval(df['aid-to-nicaraguan-contras'], confidence=0.95)

Mean: 0.5908045977011495
Low Estimate: 0.6371923522369739
High Estimate: 0.544416843165325


(0.5908045977011495, 0.544416843165325, 0.6371923522369739)

In [25]:
##calculate credible interval: bayesian approach
scipy.stats.bayes_mvs(df['aid-to-nicaraguan-contras'], alpha=0.95)

(Mean(statistic=0.5908045977011495, minmax=(0.544416843165325, 0.6371923522369739)),
 Variance(statistic=0.24343337590464026, minmax=(0.21304726488992504, 0.2780824064892091)),
 Std_dev(statistic=0.4931042359899596, minmax=(0.46157043329260705, 0.5273351936759096)))

Bayesian inference uses prior knowledge and probability to predict an outcome, whereas frequentist statistics uses concurrent trials to predict an outcome. In other words, Bayesian inference permits the statistician to use prior data to calculate future data, whereas frequentist inference only allows the statistician to analyze all the data at once and as a whole. In terms of confidence intervals, the two methods will return the same mean and standard deviation, but the philsophies are different. The Bayesian philosophy of confidence returns a range of a mean based on the given probability that that mean is accurate based on prior data points. The frequentist philosophy of confidence returns a range of a mean based adjusted to the size of the sample vs the size of the population.

## Resources

- [Worked example of Bayes rule calculation](https://en.wikipedia.org/wiki/Bayes'_theorem#Examples) (helpful as it fully breaks out the denominator)
- [Source code for mvsdist in scipy](https://github.com/scipy/scipy/blob/90534919e139d2a81c24bf08341734ff41a3db12/scipy/stats/morestats.py#L139)